### Problem 1

In [1]:
import pandas as pd
import numpy as np
import re
import gensim
import nltk
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
from gensim.models import KeyedVectors
import urllib.request

In [2]:
df = pd.read_csv("/home/kkp/Desktop/LS _ML/Week 2/Assignments/spam.csv", encoding='latin1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [33]:
import gensim.downloader as api

w2vmodel = api.load("word2vec-google-news-300")


In [34]:
def vectorize_text(text, w2v_model):
    tokens = [word for word in word_tokenize(text.lower()) if word.isalpha()]
    vectors = [w2v_model[word] for word in tokens if word in w2v_model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

In [35]:
df = pd.read_csv("spam.csv", encoding='latin-1')
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
df['v1'] = df['v1'].map({'ham': 0, 'spam': 1})
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [36]:
df.rename(columns={'v1':'label','v2':'text'},inplace=True)
df.sample(5)

,label,text
2286,0,I have had two more letters from . I will copy...
679,0,Missed your call cause I was yelling at scrapp...
2526,0,Do u noe how 2 send files between 2 computers?
3741,0,Hey i'm bored... So i'm thinking of u... So wa...
2598,0,Okie... Thanx...


In [37]:
ps = PorterStemmer()

def transform_text(text):
    tokens = nltk.word_tokenize(text.lower())
    # print(tokens)
    tokens = [token for token in tokens if token.isalnum()]
    # print(tokens)
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    # print(tokens)
    stemmed_tokens = [ps.stem(token) for token in tokens]
    # print(stemmed_tokens)

    return " ".join(stemmed_tokens)

nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /home/kkp/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [38]:
transform_text("Hi, I am KKP. I love working on Machine Learning projects and studying ML.")

'hi kkp love work machin learn project studi ml'

In [39]:
df['transformed_text']=df['text'].apply(transform_text)
df.sample(5)

,label,text,transformed_text
1370,0,Hope you are feeling great. Pls fill me in. Ab...,hope feel great pl fill abiola
5500,0,Love has one law; Make happy the person you lo...,love one law make happi person love way friend...
4599,0,Hi did u decide wot 2 get 4 his bday if not il...,hi u decid wot 2 get 4 bday ill prob ju get vo...
2124,0,Beautiful Truth against Gravity.. Read careful...,beauti truth graviti read care heart feel ligh...
5268,0,\ER,


In [49]:
X = np.vstack(df['transformed_text'].apply(lambda x: vectorize_text(x, w2vmodel)))
y = df['label']

In [50]:
df

,label,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though
...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,2nd time tri 2 contact u pound prize 2 claim e...
5568,0,Will Ì_ b going to esplanade fr home?,b go esplanad fr home
5569,0,"Pity, * was in mood for that. So...any other s...",piti mood suggest
5570,0,The guy did some bitching but I acted like i'd...,guy bitch act like interest buy someth els nex...


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

print("Accuracy:", classifier.score(X_test, y_test))
def predict_message_class(classifier, w2vmodel, message):
    transformed = transform_text(message)
    vec = vectorize_text(transformed, w2vmodel).reshape(1, -1)
    predicted = classifier.predict(vec)[0]
    return "ham" if predicted == 0 else "spam"

predict_message_class(classifier, w2vmodel, message="Nah I don't think he goes to usf, he lives around here though")

Accuracy: 0.9381165919282511


'ham'

---

### Problem 2

In [55]:
df2 = pd.read_csv("Airline-Sentiment-2-w-AA.csv")
df2

,index,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,...,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,...,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,...,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,...,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,...,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,...,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,14635,681679794,False,finalized,3,2/25/15 19:46,positive,0.3487,NaN,0.0000,...,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2/22/15 12:01,5.695880e+17,NaN,NaN
14636,14636,681679795,False,finalized,3,2/25/15 19:14,negative,1.0000,Customer Service Issue,1.0000,...,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2/22/15 11:59,5.695870e+17,Texas,NaN
14637,14637,681679796,False,finalized,3,2/25/15 19:04,neutral,1.0000,NaN,NaN,...,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2/22/15 11:59,5.695870e+17,"Nigeria,lagos",NaN
14638,14638,681679797,False,finalized,3,2/25/15 18:59,negative,1.0000,Customer Service Issue,0.6659,...,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2/22/15 11:59,5.695870e+17,New Jersey,Eastern Time (US & Canada)


In [57]:
df2 = df2[['airline_sentiment', 'text']]
df2

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [58]:
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|@\S+|#\S+", "", tweet)
    tweet = re.sub(r"[^\w\s]", "", tweet)
    tweet = contractions.fix(tweet)
    tokens = word_tokenize(tweet)
    return ' '.join([lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words('english') and w.isalpha()])


In [59]:
df2['Processed'] = df2['text'].apply(preprocess_tweet)
df2

,airline_sentiment,text,Processed
0,neutral,@VirginAmerica What @dhepburn said.,said
1,positive,@VirginAmerica plus you've added commercials t...,plus added commercial experience tacky
2,neutral,@VirginAmerica I didn't today... Must mean I n...,today must mean need take another trip
3,negative,@VirginAmerica it's really aggressive to blast...,really aggressive blast obnoxious entertainmen...
4,negative,@VirginAmerica and it's a really big bad thing...,really big bad thing
...,...,...,...
14635,positive,@AmericanAir thank you we got on a different f...,thank got different flight chicago
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,leaving minute late flight warning communicati...
14637,neutral,@AmericanAir Please bring American Airlines to...,please bring american airline
14638,negative,"@AmericanAir you have my money, you change my ...",money change flight answer phone suggestion ma...


In [61]:
X2 = np.vstack(df2['Processed'].apply(lambda x: vectorize_text(x, w2vmodel)))
y2 = df2['airline_sentiment']

In [62]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [63]:
model2 = LogisticRegression(max_iter=1000)
model2.fit(X2_train, y2_train)
print("Accuracy:", model2.score(X2_test, y2_test))

Accuracy: 0.7691256830601093


In [64]:
def predict_tweet_sentiment(model, w2vmodel, tweet):
    processed = preprocess_tweet(tweet)
    vec = vectorize_text(processed, w2vmodel).reshape(1, -1)
    return model.predict(vec)[0]

tweet = "I love flying with this airline, great service!"
predicted_sentiment = predict_tweet_sentiment(model2, w2vmodel, tweet)
print("Predicted sentiment:", predicted_sentiment)

Predicted sentiment: positive
